In [1]:
import pandas as pd
import numpy as np
import os
import logging
import esmvalcore.preprocessor
import glob
import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm
import xarray as xr
from xmip.preprocessing import rename_cmip6
import scipy.stats as stats
import matplotlib.pyplot as plt

In [2]:
### define data processsing functions which read in files and return df_s containing aerosol optical depths

def zonal_sAOD(dir, var, verbose=True):
    files=[]
    for file in os.listdir(dir):
        files.append(dir+file)
    if verbose:
        print(files)
    data = rename_cmip6(xr.open_mfdataset(files))
    data = data.load().mean(dim=['x'])
    data = data.sel(time=data['time.year']>2080).mean(dim='time')
    lats = data.y.values
    sAODs = data[var].values
    df = pd.DataFrame({'Latitude':lats, 
                       'Aerosol optical depth': sAODs})
    return df

def zonal_delta_AOD_CESM(dir):
    files=[]
    for file in os.listdir(dir):
        files.append(dir+file)
    print(files)

    control_files = []
    control_dir = dir.replace('GeoMIP', 'ScenarioMIP').replace('G6sulfur', 'ssp585').replace('p1f2', 'r1i1p1f1').replace('r2i1p1f2', 'r2i1p1f1')
    for file in os.listdir(control_dir):
        control_files.append(control_dir+file)
    print(control_files)

    data_G6 = rename_cmip6(xr.open_mfdataset(files))
    data_control = rename_cmip6(xr.open_mfdataset(control_files))
    
    data_G6 = data_G6.load().mean(dim=['x'])
    data_G6 = data_G6.sel(time=data_G6['time.year']>2080).mean(dim='time')
    lats = data_G6.y.values
    AODs_G6 = data_G6[CESM_var].values

    data_control = data_control.load().mean(dim=['x'])
    data_control = data_control.sel(time=data_control['time.year']>2080).mean(dim='time')
    AODs_control = data_control[CESM_var].values
    AODs_delta = AODs_G6 - AODs_control
    
    df = pd.DataFrame({'Latitude':lats, 
                       'Aerosol optical depth': AODs_delta})
    return df

In [3]:
##### G6 sulfur AOD

### Find UKESM and CESM runs on CEDA archive via JASMIN

UKESM_dirs = []
for x in glob.glob('/badc/cmip6/data/CMIP6/GeoMIP/*/UKESM1-0-LL/G6sulfur/*/Emon/od550aerso/*/latest/'):
    UKESM_dirs.append(x)
CESM_dirs = []
CESM_var = 'od550aer'
for x in glob.glob('/badc/cmip6/data/CMIP6/GeoMIP/*/CESM2-WACCM/G6sulfur/*/*/{c_v}/*/latest/'.format(c_v = CESM_var)):
    CESM_dirs.append(x)

CESM_control_dirs = []
for x in glob.glob('/badc/cmip6/data/CMIP6/ScenarioMIP/*/CESM2-WACCM/ssp585/*/*/{c_v}/*/latest/'.format(c_v = CESM_var)):
    CESM_control_dirs.append(x)


### add IPSL path, which was donwloaded seperately from ESGF CEDA node
IPSL_dir = 'Data/IPSL_G6sulfur/'


UKESM_dfs = []
for dir in UKESM_dirs:
    UKESM_dfs.append(zonal_sAOD(dir, var='od550aerso'))

UKESM_df = pd.concat(UKESM_dfs)
UKESM_df_mean = UKESM_df.groupby('Latitude').mean().reset_index()
UKESM_df_max = UKESM_df.groupby('Latitude').max().reset_index()
UKESM_df_min = UKESM_df.groupby('Latitude').min().reset_index()

CESM_control_dfs = []
for dir in CESM_control_dirs:
    CESM_control_dfs.append(zonal_sAOD(dir, var='od550aer'))

CESM_control_df = pd.concat(CESM_control_dfs)
CESM_control_df_mean = CESM_control_df.groupby('Latitude').mean().reset_index()


CESM_dfs = []
for dir in CESM_dirs:
    df = zonal_sAOD(dir, var='od550aer')
    out_df = pd.merge(df, CESM_control_df_mean, on='Latitude')
    out_df['delta_aod'] = out_df['Aerosol optical depth_x'] - out_df['Aerosol optical depth_y']
    CESM_dfs.append(out_df)

CESM_df = pd.concat(CESM_dfs)
CESM_df_mean = CESM_df.groupby('Latitude').mean().reset_index()
CESM_df_max = CESM_df.groupby('Latitude').max().reset_index()
CESM_df_min = CESM_df.groupby('Latitude').min().reset_index()

IPSL_df = zonal_sAOD(IPSL_dir, var='od550aerso')

['/badc/cmip6/data/CMIP6/GeoMIP/MOHC/UKESM1-0-LL/G6sulfur/r1i1p1f2/Emon/od550aerso/gn/latest/od550aerso_Emon_UKESM1-0-LL_G6sulfur_r1i1p1f2_gn_202001-204912.nc', '/badc/cmip6/data/CMIP6/GeoMIP/MOHC/UKESM1-0-LL/G6sulfur/r1i1p1f2/Emon/od550aerso/gn/latest/od550aerso_Emon_UKESM1-0-LL_G6sulfur_r1i1p1f2_gn_205001-210012.nc']
['/badc/cmip6/data/CMIP6/GeoMIP/MOHC/UKESM1-0-LL/G6sulfur/r4i1p1f2/Emon/od550aerso/gn/latest/od550aerso_Emon_UKESM1-0-LL_G6sulfur_r4i1p1f2_gn_202001-204912.nc', '/badc/cmip6/data/CMIP6/GeoMIP/MOHC/UKESM1-0-LL/G6sulfur/r4i1p1f2/Emon/od550aerso/gn/latest/od550aerso_Emon_UKESM1-0-LL_G6sulfur_r4i1p1f2_gn_205001-210012.nc']
['/badc/cmip6/data/CMIP6/GeoMIP/MOHC/UKESM1-0-LL/G6sulfur/r8i1p1f2/Emon/od550aerso/gn/latest/od550aerso_Emon_UKESM1-0-LL_G6sulfur_r8i1p1f2_gn_202001-204912.nc', '/badc/cmip6/data/CMIP6/GeoMIP/MOHC/UKESM1-0-LL/G6sulfur/r8i1p1f2/Emon/od550aerso/gn/latest/od550aerso_Emon_UKESM1-0-LL_G6sulfur_r8i1p1f2_gn_205001-210012.nc']
['/badc/cmip6/data/CMIP6/ScenarioMIP/

In [ ]:
##### 60N injection AOD

## define data paths for polar injection data
CESM_60N_data_path = 'Data/Lee_et_al/AODVISstdn_INJMAM60N_12Tg.001_map_monthly.nc'
CESM_67N_data_path = 'Data/Lee_et_al/AODVISstdn_INJMAM67N_12Tg.001_map_monthly.nc'
CESM_75N_data_path = 'Data/Lee_et_al/AODVISstdn_INJMAM75N_12Tg.001_map_monthly.nc'
UKESM_data_path = 'Data/UKESM_60N/u-cm174_60N_od550aer_zonal.nc'

def process_CESM(path):
    CESM_sod = xr.open_dataset(path)
    CESM_sod = CESM_sod.mean(dim=['lon', 'time'])
    
    CESM_od_df = pd.DataFrame({'Latitude':CESM_sod.lat,
                                'saod': CESM_sod.AODVISstdn})
    return CESM_od_df

CESM_60N_od_df = process_CESM(CESM_60N_data_path)
CESM_67N_od_df = process_CESM(CESM_67N_data_path)
CESM_75N_od_df = process_CESM(CESM_75N_data_path)

UKESM_od = xr.open_dataset(UKESM_data_path)
UKESM_od = UKESM_od.mean(dim=['time'])

#need to compare to ssp245 because stratopsheric od variable isn't available
UKESM_control_dirs = []
for x in glob.glob('/badc/cmip6/data/CMIP6/ScenarioMIP/*/UKESM1-0-LL/ssp245/*/AERmon/od550aer/*/latest/'):
    UKESM_control_dirs.append(x)
UKESM_control_dfs = []
for dir in UKESM_control_dirs:
    UKESM_control_dfs.append(zonal_sAOD(dir, var='od550aer', verbose=False))

UKESM_control_df = pd.concat(UKESM_control_dfs)
UKESM_control_df_mean = UKESM_control_df.groupby('Latitude').mean().reset_index()

var = 'atmosphere_optical_thickness_due_to_ambient_aerosol_particles__550nm'
UKESM_od_df = pd.DataFrame({'Latitude':UKESM_od.latitude,
                            'aod': UKESM_od[var]})
UKESM_60N_od_df = pd.merge(UKESM_control_df_mean, UKESM_od_df, on='Latitude')
UKESM_60N_od_df['delta_aod'] = UKESM_60N_od_df['aod'] - UKESM_60N_od_df['Aerosol optical depth']
UKESM_norm_factor = np.max(CESM_60N_od_df['saod'])/np.max(UKESM_60N_od_df['delta_aod'])
UKESM_60N_od_df['normalised_delta_aod'] = UKESM_60N_od_df['delta_aod']*UKESM_norm_factor

In [ ]:
######### PLOT 

fig, (ax1, ax2) = plt.subplots(1,2,figsize=(9,4), sharey=True)

plt.rcParams['font.size'] = '12'

ax1.plot(UKESM_df_mean['Latitude'], UKESM_df_mean['Aerosol optical depth'], c='blue', 
         lw=1.5, label='UKESM (3)')
ax1.fill_between(UKESM_df_max['Latitude'], UKESM_df_min['Aerosol optical depth'],
                 UKESM_df_max['Aerosol optical depth'], color='b', alpha=0.3)

ax1.plot(CESM_df_mean['Latitude'], CESM_df_mean['delta_aod'], c='g', 
         lw=1.5, label='CESM (2)')
ax1.fill_between(CESM_df_max['Latitude'], CESM_df_min['delta_aod'],
                 CESM_df_max['delta_aod'], color='g', alpha=0.3)


ax1.plot(IPSL_df['Latitude'], IPSL_df['Aerosol optical depth'], c='red', 
         lw=1.5, label='IPSL (1)')
     
ax1.legend(fontsize='medium', loc='upper right')
ax1.set_xlim(-90,90)
ax1.set_ylim(0,0.85)
ax1.set_xticks([-90, -60, -30, 0, 30, 60, 90])

ax2.plot(CESM_60N_od_df['Latitude'], CESM_60N_od_df['saod'], c='green', label = '60N')
ax2.plot(CESM_67N_od_df['Latitude'], CESM_67N_od_df['saod'], c='green', ls='--', label = '67N')
ax2.plot(CESM_75N_od_df['Latitude'], CESM_75N_od_df['saod'], c='green', ls=':', label = '75N')
ax2.axvline(x = 60, ymin=0, ymax=0.132, color = 'black')
ax2.axvline(x = 67, ymin=0, ymax=0.132, color = 'black', ls='--')
ax2.axvline(x = 75, ymin=0, ymax=0.132, color = 'black', ls=':')

ax2.legend(fontsize='medium')
ax2.set_xlim(-90,90)
ax2.set_xticks([-90, -60, -30, 0, 30, 60, 90])

ax1.set_ylabel('Aerosol optical depth', fontsize='large')
ax1.set_title('Equatorial injection', fontsize='large')
ax2.set_title('Arctic injection', fontsize='large')
ax1.annotate('(a)', (-85, 0.79), fontsize='large')
ax2.annotate('(b)', (-85, 0.79), fontsize='large')

plt.tight_layout()


plt.savefig('Figures/sAOD_comaprison.png', dpi=300)
plt.show()
